In [2]:
!pip install https://github.com/p-koo/tfomics/tarball/master

  Using cached https://github.com/p-koo/tfomics/tarball/master


In [2]:
import tensorflow as tf

import numpy as np
import requests as rq
import io, h5py, os

from tfomics import moana, evaluate

Matplotlib is building the font cache; this may take a moment.


In [3]:
data = rq.get('https://www.dropbox.com/s/c3umbo5y13sqcfp/synthetic_dataset.h5?raw=true')
data.raise_for_status()

with h5py.File(io.BytesIO(data.content), 'r') as dataset:
    x_train = np.array(dataset['X_train']).astype(np.float32).transpose([0, 2, 1])
    y_train = np.array(dataset['Y_train']).astype(np.float32)
    x_valid = np.array(dataset['X_valid']).astype(np.float32).transpose([0, 2, 1])
    y_valid = np.array(dataset['Y_valid']).astype(np.int32)
    x_test = np.array(dataset['X_test']).astype(np.float32).transpose([0, 2, 1])
    y_test = np.array(dataset['Y_test']).astype(np.int32)

In [4]:
directories = [i[0] for i in os.walk('models')][1:]
models = [i[2] for i in os.walk('models')][1:]

In [33]:
for i in range(len(directories)):
    print(i / len(directories) * 100)
    for j in range(len(models[i])):
        
        model = tf.keras.models.load_model(h5py.File(os.path.join(directories[i], models[i][j]), 'r'))
        index = [type(i) for i in model.layers].index(tf.keras.layers.Activation)
        
        ppms = moana.filter_activations(x_test, model, layer=index, window=20, threshold=0.5)
        ppms = moana.clip_filters(ppms, threshold=0.5, pad=3)
        
        if not os.path.exists(os.path.join('motifs', directories[i].split('\\')[1])):
            os.makedirs(os.path.join('motifs', directories[i].split('\\')[1]))
        
        moana.meme_generate(ppms, output_file=os.path.join('motifs', directories[i].split('\\')[1], models[i][j].split('.h5')[0] + '.meme'), prefix='filter')

0.0
8.333333333333332
16.666666666666664
25.0
33.33333333333333
41.66666666666667
50.0
58.333333333333336
66.66666666666666
75.0
83.33333333333334
91.66666666666666
